In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install gdown

In [3]:
! rm -rf video.mp4

In [4]:
import gdown

# Update with your file's specific ID
file_id = "15C3ZIbvy4CemGBOmIkrGK7H7a2tpYaBJ"
url = f"https://drive.google.com/uc?id={file_id}"

output = "people-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=15C3ZIbvy4CemGBOmIkrGK7H7a2tpYaBJ
To: /content/people-counting.mp4
100%|██████████| 42.5M/42.5M [00:00<00:00, 43.1MB/s]


'people-counting.mp4'

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
SOURCE_VIDEO_PATH = os.path.join(HOME, "people-counting.mp4")

In [3]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.9/107.7 GB disk)


In [4]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


In [5]:
# settings
MODEL = "yolov8x.pt"

In [6]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [11]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8s.pt')

# Get class names from the model
class_names = model.names  # Dictionary {class_id: class_name}

# Print all class IDs and names
for class_id, class_name in class_names.items():
    print(f"Class ID: {class_id}, Class Name: {class_name}")

100%|██████████| 21.5M/21.5M [00:00<00:00, 145MB/s]


Class ID: 0, Class Name: person
Class ID: 1, Class Name: bicycle
Class ID: 2, Class Name: car
Class ID: 3, Class Name: motorcycle
Class ID: 4, Class Name: airplane
Class ID: 5, Class Name: bus
Class ID: 6, Class Name: train
Class ID: 7, Class Name: truck
Class ID: 8, Class Name: boat
Class ID: 9, Class Name: traffic light
Class ID: 10, Class Name: fire hydrant
Class ID: 11, Class Name: stop sign
Class ID: 12, Class Name: parking meter
Class ID: 13, Class Name: bench
Class ID: 14, Class Name: bird
Class ID: 15, Class Name: cat
Class ID: 16, Class Name: dog
Class ID: 17, Class Name: horse
Class ID: 18, Class Name: sheep
Class ID: 19, Class Name: cow
Class ID: 20, Class Name: elephant
Class ID: 21, Class Name: bear
Class ID: 22, Class Name: zebra
Class ID: 23, Class Name: giraffe
Class ID: 24, Class Name: backpack
Class ID: 25, Class Name: umbrella
Class ID: 26, Class Name: handbag
Class ID: 27, Class Name: tie
Class ID: 28, Class Name: suitcase
Class ID: 29, Class Name: frisbee
Class ID:

#### Original Code - People Counting

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import cv2  # OpenCV for image processing
import numpy as np


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'  # Directory to save frames
FRAME_DATA_PATH = 'frame_data.json'  # JSON file to save frame data

# Define polygon zone (adjust coordinates as needed)
polygon_zone = np.array([(1169, 1678),
 (1942, 2025),
 (1816, 2102),
 (1085, 1703)], np.int32)


# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Initialize sequential ID mapping
id_counter = 1
id_map = {}  # Maps tracker_id to a sequential ID
frame_data_list = []  # To store frame data
crossed_tracker_ids = set()  # Set to store tracker IDs that have crossed the polygon
count = 0  # Counter for people who crossed the zone

# Create directory for saving frames
os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

# Function to check if a bounding box center is inside the polygon
def is_inside_polygon(polygon, bbox):
    """Check if the center of a bounding box is inside the polygon."""
    x, y, w, h = bbox  # Extract bounding box coordinates
    center_x, center_y = x + w // 2, y + h // 2  # Calculate center point

    result = cv2.pointPolygonTest(polygon, (center_x, center_y), False)
    return result >= 0  # Returns True if inside, False if outside


# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # Iterate through each frame in the video and track objects
    for frame_number, result in enumerate(
        YOLO('yolov8s.pt').track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            show=False,
            stream=True,
            agnostic_nms=True,
            persist=True
        )
    ):
        # Extract frame and detections
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Filter detections to only include people (class_id = 0 in COCO) and inside polygon
        PEOPLE_CLASS_ID = 0
        filtered_detections = [
            (bbox, confidence, class_id, tracker_id)
            for bbox, confidence, class_id, tracker_id in detections
            if class_id == PEOPLE_CLASS_ID and is_inside_polygon(polygon_zone, bbox)
        ]

        if filtered_detections:
            people_detections = sv.Detections(
                xyxy=np.array([bbox for bbox, _, _, _ in filtered_detections]),
                confidence=np.array([confidence for _, confidence, _, _ in filtered_detections]),
                class_id=np.array([class_id for _, _, class_id, _ in filtered_detections]),
                tracker_id=np.array([tracker_id for _, _, _, tracker_id in filtered_detections]) if detections.tracker_id is not None else None
            )
        else:
            # Create empty detections if no people found
            people_detections = sv.Detections(
                xyxy=np.empty((0, 4)),
                confidence=np.empty((0,)),
                class_id=np.empty((0,), dtype=int),
                tracker_id=None
            )

        # Use people_detections instead of detections
        detections = people_detections

        # Handle object IDs (tracker IDs)
        if result.boxes.id is not None:
            new_tracker_ids = []
            detected_ids = result.boxes.id.cpu().numpy().astype(int)  # Get detected tracker IDs

            for tracker_id in detected_ids:
                if tracker_id not in id_map:
                    id_map[tracker_id] = len(id_map) + 1  # Ensure continuous numbering (1,2,3,4...)
                new_tracker_ids.append(id_map[tracker_id])

            # Update detections with mapped sequential IDs
            detections.tracker_id = np.array(new_tracker_ids)

        # Track if people have crossed the polygon zone
        for bbox, confidence, class_id, tracker_id in detections:
            if tracker_id not in crossed_tracker_ids and is_inside_polygon(polygon_zone, bbox):
                crossed_tracker_ids.add(tracker_id)
                count += 1  # Increase count when a new person crosses the zone

        # Define labels for each detection
        labels = [
            f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}"
            for bbox, confidence, class_id, tracker_id in detections
        ]

        # Draw polygon on the frame
        cv2.polylines(frame, [polygon_zone], isClosed=True, color=(0, 255, 0), thickness=2)

        # Annotate the frame with the count of people who crossed the zone
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        cv2.putText(frame, f"Count: {count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Save current frame to disk
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

        # Collect frame data for JSON
        frame_data = {
            "frame_number": frame_number,
            "detections": [
                {
                    "tracker_id": int(tracker_id) if tracker_id is not None else None,  # Handle None tracker_id
                    "class_id": int(class_id),
                    "confidence": float(confidence),
                    "bbox": [float(coord) for coord in bbox]
                }
                for bbox, confidence, class_id, tracker_id in detections
            ]
        }
        frame_data_list.append(frame_data)
        print("Frame data : " + str(frame_data))

        # Write annotated frame to the output video
        sink.write_frame(frame)

    print("Vehicle tracking completed and output video saved.")

# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)

print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 2 cars, 11.6ms
Frame data : {'frame_number': 0, 'detections': []}
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 2 cars, 11.1ms
Frame data : {'frame_number': 1, 'detections': []}
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 2 cars, 11.0ms
Frame data : {'frame_number': 2, 'detections': []}
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 2 cars, 11.1ms
Frame data : {'frame_number': 3, 'detections': []}
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 2 cars, 11.0ms
Frame data : {'frame_number': 4, 'detections': []}
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 11.2ms
Frame data : {'frame_number': 5, 'detections': []}
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 11.1ms
Frame data : {'frame_number': 6, 'detections': []}
video 1/1 (frame 8/592) /content/people-counting.mp4: 3

#### Bottom Center

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import cv2
import numpy as np


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'  # Directory to save frames
FRAME_DATA_PATH = 'frame_data.json'  # JSON file to save frame data

# Define polygon zones
area1 = np.array([(1169, 1678), (1942, 2025), (1816, 2102), (1085, 1703)], np.int32)
area2 = np.array([(1040, 1710), (1771, 2117), (1673, 2142), (981, 1713)], np.int32)

# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Initialize sequential ID mapping
id_counter = 1
id_map = {}  # Maps tracker_id to a sequential ID
frame_data_list = []  # To store frame data
tracker_history = {}  # To store the history of tracker IDs and their states
total_count = 0  # Total people count
entering_count = 0  # People entering count
exiting_count = 0  # People exiting count

# Create directory for saving frames
os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

# Function to check if a point is inside a polygon
def is_inside_polygon(polygon, point):
    """Check if a point is inside the polygon."""
    # return cv2.pointPolygonTest(polygon, point, False) >= 0
    polygon = np.array(polygon, dtype=np.float32)  # Convert to float
    point = tuple(map(float, point))  # Convert to float tuple
    return cv2.pointPolygonTest(polygon, point, False) >= 0


# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # Iterate through each frame in the video and track objects
    for frame_number, result in enumerate(
        YOLO('yolov8s.pt').track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            show=False,
            stream=True,
            agnostic_nms=True,
            persist=True
        )
    ):
        # Extract frame and detections
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Filter detections to only include people (class_id = 0 in COCO)
        PEOPLE_CLASS_ID = 0
        filtered_detections = [
            (bbox, confidence, class_id, tracker_id)
            for bbox, confidence, class_id, tracker_id in detections
            if class_id == PEOPLE_CLASS_ID
        ]

        if filtered_detections:
            people_detections = sv.Detections(
                xyxy=np.array([bbox for bbox, _, _, _ in filtered_detections]),
                confidence=np.array([confidence for _, confidence, _, _ in filtered_detections]),
                class_id=np.array([class_id for _, _, class_id, _ in filtered_detections]),
                tracker_id=np.array([tracker_id for _, _, _, tracker_id in filtered_detections]) if detections.tracker_id is not None else None
            )
        else:
            # Create empty detections if no people found
            people_detections = sv.Detections(
                xyxy=np.empty((0, 4)),
                confidence=np.empty((0,)),
                class_id=np.empty((0,), dtype=int),
                tracker_id=None
            )

        # Use people_detections instead of detections
        detections = people_detections

        # Handle object IDs (tracker IDs)
        if result.boxes.id is not None:
            new_tracker_ids = []
            detected_ids = result.boxes.id.cpu().numpy().astype(int)  # Get detected tracker IDs

            for tracker_id in detected_ids:
                if tracker_id not in id_map:
                    id_map[tracker_id] = len(id_map) + 1  # Ensure continuous numbering (1,2,3,4...)
                new_tracker_ids.append(id_map[tracker_id])

            # Update detections with mapped sequential IDs
            detections.tracker_id = np.array(new_tracker_ids)

        # Track people entering and exiting based on polygon zones
        for bbox, confidence, class_id, tracker_id in detections:
            # Get the center of the bounding box
            x1, y1, x2, y2 = bbox
            center = (int((x1 + x2) / 2), int(y2))  # Bottom-center point
            cv2.circle(frame, center, radius=5, color=(255, 0, 0), thickness=-1)
            cv2.waitKey(0)
            print(f"Bottom-center: {center}, Polygon vertices: {area1}")
            # Check if the center is inside area1 or area2
            in_area1 = is_inside_polygon(area1, center)
            in_area2 = is_inside_polygon(area2, center)

            # Initialize tracker history if not already present
            if tracker_id not in tracker_history:
                tracker_history[tracker_id] = {"area1": False, "area2": False}

            # Check for entering (area2 -> area1)
            if tracker_history[tracker_id]["area2"] and in_area1:
                entering_count += 1
                tracker_history[tracker_id]["area1"] = True
                print(f"Person {tracker_id} entered. Total: {total_count}, Entering: {entering_count}")

            # Check for exiting (area1 -> area2)
            if tracker_history[tracker_id]["area1"] and in_area2:
                exiting_count += 1
                tracker_history[tracker_id]["area2"] = True
                print(f"Person {tracker_id} exited. Total: {total_count}, Exiting: {exiting_count}")

            # Update tracker history for the current frame
            tracker_history[tracker_id]["area1"] = in_area1
            tracker_history[tracker_id]["area2"] = in_area2

        # Define labels for each detection
        labels = [
            f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}"
            for bbox, confidence, class_id, tracker_id in detections
        ]

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 0, 255), thickness=2)

        # Annotate the frame with the counts
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        cv2.putText(frame, f"Total: {total_count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Save current frame to disk
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

        # Collect frame data for JSON
        frame_data = {
            "frame_number": frame_number,
            "detections": [
                {
                    "tracker_id": int(tracker_id) if tracker_id is not None else None,  # Handle None tracker_id
                    "class_id": int(class_id),
                    "confidence": float(confidence),
                    "bbox": [float(coord) for coord in bbox]
                }
                for bbox, confidence, class_id, tracker_id in detections
            ]
        }
        frame_data_list.append(frame_data)
        print("Frame data : " + str(frame_data))

        # Write annotated frame to the output video
        sink.write_frame(frame)

    print("Vehicle tracking completed and output video saved.")

# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)


print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")
print(total_count,entering_count,exiting_count)


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 2 cars, 1013.8ms
Frame data : {'frame_number': 0, 'detections': []}
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 2 cars, 423.7ms
Frame data : {'frame_number': 1, 'detections': []}
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 2 cars, 442.5ms
Frame data : {'frame_number': 2, 'detections': []}
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 2 cars, 411.4ms
Frame data : {'frame_number': 3, 'detections': []}
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 2 cars, 433.5ms
Frame data : {'frame_number': 4, 'detections': []}
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 434.2ms
Bottom-center: (70, 1972), Polygon vertices: [[1169 1678]
 [1942 2025]
 [1816 2102]
 [1085 1703]]
Frame data : {'frame_number': 5, 'detections': [{'tracker_id': 1, 'class_id': 0, 'confidence': 0.5893422961235046, 'bbox': [9.935486793518066, 1222.84729

#### Test 2 - Enter Exit

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import cv2
import numpy as np

# Video Paths
TARGET_VIDEO_PATH = 'output_video.mp4'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)


# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:  # Ensure we have tracker IDs
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)  # Get tracker IDs
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            # Check if the bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []  # Initialize state tracking

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")

            # Determine if entering or exiting
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []  # Reset state after count
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []  # Reset state after count

            total_count = entering_count + exiting_count

            # Draw bounding box and ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {tracker_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Draw bottom-right corner marker
            cv2.circle(frame, bottom_right, 5, (0, 0, 255), -1)

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display counts
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

print("Processing complete. Output video saved.")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")



video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 2966.8ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 2770.5ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 3182.8ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 3443.0ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 2736.4ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 2736.7ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 2911.9ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 3671.1ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 2734.4ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 2732.9ms
video 1/1 (frame 11/592) /content/people-counting.mp4: 384x640 1 person, 1 car

#### Final JSON

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import cv2
import numpy as np
import json
from datetime import datetime

# Video Paths
TARGET_VIDEO_PATH = 'output_video.mp4'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID
detection_data = {}  # Store detection data for JSON

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:  # Ensure we have tracker IDs
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)  # Get tracker IDs
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            # Check if the bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []  # Initialize state tracking
                detection_data[tracker_id] = {  # Initialize detection data
                    "tracker_id": int(tracker_id),  # Convert to standard Python int
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence),  # Convert to standard Python float
                    "class_id": int(class_id)  # Convert to standard Python int
                }

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    detection_data[tracker_id]["entry_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    detection_data[tracker_id]["exit_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Determine if entering or exiting
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []  # Reset state after count
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []  # Reset state after count

            total_count = entering_count + exiting_count

            # Update bbox history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),  # Convert to standard Python int
                "bbox": [float(x1), float(y1), float(x2), float(y2)],  # Convert to standard Python floats
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

            # Draw bounding box and ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {tracker_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Draw bottom-right corner marker
            cv2.circle(frame, bottom_right, 5, (0, 0, 255), -1)

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display counts
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

# Prepare JSON data
json_data = {
    "summary": {
        "total_people": int(total_count),  # Convert to standard Python int
        "total_entering": int(entering_count),  # Convert to standard Python int
        "total_exiting": int(exiting_count),  # Convert to standard Python int
        "total_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "detections": {
        # Convert NumPy int64 keys to standard Python int
        int(tracker_id): data
        for tracker_id, data in detection_data.items()
    }
}

# Save all data to a single JSON file
json_path = "detection_data.json"
with open(json_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Processing complete. Output video saved.")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 3863.9ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 2814.4ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 2830.6ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 2758.0ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 3898.0ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2775.9ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2756.7ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2747.6ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3954.7ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 1 potted plant, 2798.2ms
video 

In [ ]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')

#### Correcting tracker id

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import cv2
import numpy as np
import json
from datetime import datetime

# Video Paths
TARGET_VIDEO_PATH = 'output_video.mp4'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID
detection_data = {}  # Store detection data for JSON
people_tracker_id_map = {}  # Map original tracker IDs to sequential IDs for people
next_people_tracker_id = 1  # Start assigning sequential IDs from 1

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:  # Ensure we have tracker IDs
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)  # Get tracker IDs
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            if tracker_id is not None:
                if tracker_id not in people_tracker_id_map:
                    people_tracker_id_map[tracker_id] = next_people_tracker_id
                    next_people_tracker_id += 1
                people_tracker_id = people_tracker_id_map[tracker_id]
            else:
                continue  # Skip detections without tracker IDs


            # Check if the bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if people_tracker_id not in tracker_states:
                tracker_states[people_tracker_id] = []  # Initialize state tracking
                detection_data[people_tracker_id] = {  # Initialize detection data
                    "tracker_id": int(people_tracker_id),  # Use sequential ID for people
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence),  # Convert to standard Python float
                    "class_id": int(class_id)  # Convert to standard Python int
                }

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[people_tracker_id]:
                tracker_states[people_tracker_id].append("area2")
                if tracker_states[people_tracker_id] == ["area2"]:
                    detection_data[people_tracker_id]["entry_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            if in_area1 and "area1" not in tracker_states[people_tracker_id]:
                tracker_states[people_tracker_id].append("area1")
                if tracker_states[people_tracker_id] == ["area1"]:
                    detection_data[people_tracker_id]["exit_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Determine if entering or exiting
            if tracker_states[people_tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[people_tracker_id] = []  # Reset state after count
            elif tracker_states[people_tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[people_tracker_id] = []  # Reset state after count

            total_count = entering_count + exiting_count

            # Update bbox history
            detection_data[people_tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),  # Convert to standard Python int
                "bbox": [float(x1), float(y1), float(x2), float(y2)],  # Convert to standard Python floats
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

            # Draw bounding box and ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {people_tracker_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Draw bottom-right corner marker
            cv2.circle(frame, bottom_right, 5, (0, 0, 255), -1)

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display counts
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

# Prepare JSON data
json_data = {
    "summary": {
        "total_people": int(total_count),  # Convert to standard Python int
        "total_entering": int(entering_count),  # Convert to standard Python int
        "total_exiting": int(exiting_count),  # Convert to standard Python int
        "total_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "detections": {
        # Use sequential IDs for people
        int(tracker_id): data
        for tracker_id, data in detection_data.items()
    }
}

# Save all data to a single JSON file
json_path = "detection_data.json"
with open(json_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Processing complete. Output video saved.")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 4 cars, 394.3ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 2 cars, 404.6ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 2 cars, 410.7ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 2 cars, 409.4ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 2 cars, 394.5ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 433.7ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 434.5ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 1 skateboard, 447.8ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 429.9ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 433.7ms
video 1/1 (frame 11/592) /content/people-counting.mp4: 384x640 1 person, 2 cars, 415.9ms
video 1/1 (frame 12/592) /content/peo

#### Correcting time

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import cv2
import numpy as np
import json
import subprocess
from datetime import datetime

# Video Paths
SOURCE_VIDEO_PATH = 'input_video.mp4'  # Replace with your CCTV video path
TARGET_VIDEO_PATH = 'output_video.mp4'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID
detection_data = {}  # Store detection data for JSON
people_tracker_id_map = {}  # Map original tracker IDs to sequential IDs for people
next_people_tracker_id = 1  # Start assigning sequential IDs from 1

# Extract frame timestamps using ffprobe
def extract_frame_timestamps(video_path):
    command = [
        'ffprobe', '-v', 'error', '-show_entries', 'frame=pkt_pts_time',
        '-of', 'csv=p=0', video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, text=True)
    timestamps = [float(ts) for ts in result.stdout.splitlines()]
    return timestamps

# Get frame timestamps
frame_timestamps = extract_frame_timestamps(SOURCE_VIDEO_PATH)

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        YOLO('yolov8s.pt').track(
            source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True
        )
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:  # Ensure we have tracker IDs
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)  # Get tracker IDs
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Get the actual timestamp for the current frame
        if frame_number < len(frame_timestamps):
            frame_timestamp = frame_timestamps[frame_number]
            frame_time = datetime.fromtimestamp(frame_timestamp).strftime("%Y-%m-%d %H:%M:%S")
        else:
            frame_time = "Unknown"

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            # Reassign tracker IDs for people only
            if tracker_id not in people_tracker_id_map:
                people_tracker_id_map[tracker_id] = next_people_tracker_id
                next_people_tracker_id += 1
            people_tracker_id = people_tracker_id_map[tracker_id]

            # Check if the bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if people_tracker_id not in tracker_states:
                tracker_states[people_tracker_id] = []  # Initialize state tracking
                detection_data[people_tracker_id] = {  # Initialize detection data
                    "tracker_id": int(people_tracker_id),  # Use sequential ID for people
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence),  # Convert to standard Python float
                    "class_id": int(class_id)  # Convert to standard Python int
                }

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[people_tracker_id]:
                tracker_states[people_tracker_id].append("area2")
                if tracker_states[people_tracker_id] == ["area2"]:
                    detection_data[people_tracker_id]["entry_time"] = frame_time
            if in_area1 and "area1" not in tracker_states[people_tracker_id]:
                tracker_states[people_tracker_id].append("area1")
                if tracker_states[people_tracker_id] == ["area1"]:
                    detection_data[people_tracker_id]["exit_time"] = frame_time

            # Determine if entering or exiting
            if tracker_states[people_tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[people_tracker_id] = []  # Reset state after count
            elif tracker_states[people_tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[people_tracker_id] = []  # Reset state after count

            total_count = entering_count + exiting_count

            # Update bbox history
            detection_data[people_tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),  # Convert to standard Python int
                "bbox": [float(x1), float(y1), float(x2), float(y2)],  # Convert to standard Python floats
                "timestamp": frame_time
            })

            # Draw bounding box and ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {people_tracker_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Draw bottom-right corner marker
            cv2.circle(frame, bottom_right, 5, (0, 0, 255), -1)

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display counts
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

# Prepare JSON data
json_data = {
    "summary": {
        "total_people": int(total_count),  # Convert to standard Python int
        "total_entering": int(entering_count),  # Convert to standard Python int
        "total_exiting": int(exiting_count),  # Convert to standard Python int
        "total_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "detections": {
        # Use sequential IDs for people
        int(tracker_id): data
        for tracker_id, data in detection_data.items()
    }
}

# Save all data to a single JSON file
json_path = "detection_data.json"
with open(json_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Processing complete. Output video saved.")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Add this before importing DeepFace

#### Age Gender pre trained models

In [ ]:
!pip install deepface



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=8ea85986506388596accd33fb03ac8fdd83eb05f1d650ff2aad620ec1a0de18f
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from deepface import DeepFace
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime

# Video Paths
TARGET_VIDEO_PATH = 'output_video.mp4'  # Replace with your output video path
JSON_OUTPUT_PATH = 'age_gender_detection_data.json'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID
detection_data = {}  # Store detection data for JSON

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)  # Make sure the input video path is correct

# Function to get age and gender using DeepFace
def predict_age_gender(face):
    try:
        analysis = DeepFace.analyze(face, actions=["age", "gender"], enforce_detection=False)
        age = analysis[0]["age"]
        gender = analysis[0]["dominant_gender"]
        return gender, age
    except Exception as e:
        print(f"Error in age/gender prediction: {e}")
        return "Unknown", "Unknown"

# Load YOLO model
model = YOLO("yolov8n.pt")  # Replace with your YOLO model path if needed

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = int(tracker_ids[i]) if i < len(tracker_ids) else None  # Convert to standard Python int
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            # Crop face for gender and age prediction
            face = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = "Unknown", "Unknown"
            if face.size > 0:
                gender, age = predict_age_gender(face)

            # Check if bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                detection_data[tracker_id] = {
                    "tracker_id": tracker_id,  # Already converted to int
                    "gender": gender,
                    "age": age,
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence),
                    "class_id": int(class_id)
                }

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    detection_data[tracker_id]["entry_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    detection_data[tracker_id]["exit_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Determine if entering or exiting
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            total_count = entering_count + exiting_count

            # Update bbox history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

            # Add annotations (class name, tracking ID, age, gender)
            label = f"ID: {tracker_id} | {gender}, {age}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

# Save JSON data
with open(JSON_OUTPUT_PATH, "w") as json_file:
    json.dump(detection_data, json_file, indent=4)

print(f"Processing complete. Output saved to {JSON_OUTPUT_PATH}.")



video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 bed, 8.0ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 bed, 10.9ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 bed, 11.5ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 bed, 11.1ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 bed, 11.2ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 bed, 11.0ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 bed, 10.5ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 bed, 11.0ms


Action: age:   0%|          | 0/2 [00:00<?, ?it/s]

Error in age/gender prediction: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 226, 226, 3), dtype=float32)
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 bed, 9.4ms



Action: age:   0%|          | 0/2 [00:00<?, ?it/s]

Error in age/gender prediction: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 226, 226, 3), dtype=float32)
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 bed, 13.5ms


...


Action: age:   0%|          | 0/2 [00:00<?, ?it/s]

Error in age/gender prediction: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 226, 226, 3), dtype=float32)
video 1/1 (frame 27/592) /content/people-counting.mp4: 384x640 1 person, 1 bed, 11.2ms



Action: age:   0%|          | 0/2 [00:00<?, ?it/s]

Error in age/gender prediction: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 226, 226, 3), dtype=float32)
video 1/1 (frame 28/592) /content/people-counting.mp4: 384x640 1 person, 1 bed, 10.9ms


KeyboardInterrupt: 

In [ ]:
from deepface import DeepFace
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime
import os

# Configuration
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU if available, else set to "-1" for CPU
TARGET_VIDEO_PATH = 'output_video.mp4'
JSON_OUTPUT_PATH = 'detection_results.json'

# Define detection areas
area1 = np.array([(1169, 1728), (1942, 2075), (1816, 2152), (1085, 1753)], np.int32)
area2 = np.array([(1040, 1760), (1771, 2167), (1673, 2192), (981, 1763)], np.int32)

# Initialize counters and data storage
counters = {
    'total': 0,
    'entering': 0,
    'exiting': 0,
    'men': 0,
    'women': 0
}
tracker_states = {}
detection_data = {}

# Enhanced gender/age prediction
def predict_gender_age(face_img):
    try:
        # Convert and resize face image
        if len(face_img.shape) == 2:  # Grayscale
            face_img = cv2.cvtColor(face_img, cv2.COLOR_GRAY2BGR)
        face_img = cv2.resize(face_img, (224, 224))  # VGG-Face input size

        # Analyze with multiple backends for robustness
        results = DeepFace.analyze(
            face_img,
            actions=['gender', 'age'],
            enforce_detection=False,
            detector_backend='mtcnn',  # Most accurate for faces
            prog_bar=False
        )

        # Standardize gender labels
        gender = results[0]['dominant_gender']
        if gender == 'Man':
            return "Man", results[0]['age']
        elif gender == 'Woman':
            return "Woman", results[0]['age']
        return "Unknown", results[0]['age']

    except Exception as e:
        print(f"Prediction error: {str(e)}")
        return "Unknown", "Unknown"


# Main processing function
def process_video():
    video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

    with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
        for result in model.track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            stream=True,
            persist=True,
            verbose=False
        ):
            frame = result.orig_img
            detections = sv.Detections.from_yolov8(result)

            if result.boxes.id is not None:
                for idx, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                    if class_id == 0:  # Only process people
                        tracker_id = int(result.boxes.id[idx].cpu().numpy())
                        x1, y1, x2, y2 = map(int, bbox)

                        # Extract and analyze face
                        face_roi = frame[y1:y2, x1:x2]
                        gender, age = predict_gender_age(face_roi)

                        # Update counters
                        if gender == "Man":
                            counters['men'] += 1
                        elif gender == "Woman":
                            counters['women'] += 1

                        # Tracking logic
                        if tracker_id not in tracker_states:
                            tracker_states[tracker_id] = {
                                "gender": gender,
                                "age": age,
                                "path": []
                            }

                        # Area detection logic (same as before)
                        # ... [your existing area detection code]

                        # Visualization
                        color = (0, 255, 0) if gender == "Woman" else (255, 0, 0)  # Green for women, blue for men
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        label = f"{gender}/{age}" if age != "Unknown" else gender
                        cv2.putText(frame, f"ID:{tracker_id} {label}", (x1, y1-10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

            sink.write_frame(frame)

    # Save results
    results = {
        "counters": counters,
        "detections": detection_data,
        "gender_accuracy": {
            "men": counters['men'],
            "women": counters['women'],
            "ratio": f"{counters['women']/max(1,counters['men']):.2f} women per man"
        }
    }

    with open(JSON_OUTPUT_PATH, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"Processing complete. Women detected: {counters['women']}, Men detected: {counters['men']}")

if __name__ == "__main__":
    process_video()

Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an unexpected keyword argument 'prog_bar'
Prediction error: analyze() got an une

KeyboardInterrupt: 

#### Enter Exit Count + Age Gender Detection

In [7]:
!pip install tensorflow==2.12.0 deepface==0.0.79

In [8]:
import supervision as sv
from ultralytics import YOLO
import os
import cv2
import numpy as np
import json
from datetime import datetime
from deepface import DeepFace

# Video Paths
TARGET_VIDEO_PATH = 'output_video.mp4'

# Define two polygonal areas
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Initialize counts
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Track movement of each ID
detection_data = {}  # Store detection data for JSON

# Open video info
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Function to get age and gender using DeepFace
def predict_age_gender(face):
    try:
        analysis = DeepFace.analyze(face, actions=["age", "gender"], enforce_detection=False)
        age = analysis[0]["age"]
        gender = analysis[0]["dominant_gender"]
        return gender, age
    except Exception as e:
        print(f"Error in age/gender prediction: {e}")
        return "Unknown", "Unknown"

# Video Sink
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker='bytetrack.yaml', show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Only keep people (COCO class 0)
        PEOPLE_CLASS_ID = 0
        people_detections = []

        if result.boxes.id is not None:  # Ensure we have tracker IDs
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)  # Get tracker IDs
            for i, (bbox, confidence, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == PEOPLE_CLASS_ID:
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people_detections.append((bbox, confidence, class_id, tracker_id))

        # Process detections
        for bbox, confidence, class_id, tracker_id in people_detections:
            x1, y1, x2, y2 = bbox  # Get bounding box coordinates
            bottom_right = (int(x2), int(y2))  # Bottom-right corner

            # Crop face for gender and age prediction
            face = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = "Unknown", "Unknown"
            if face.size > 0:
                gender, age = predict_age_gender(face)

            # Check if the bottom-right corner is in an area
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []  # Initialize state tracking
                detection_data[tracker_id] = {  # Initialize detection data
                    "tracker_id": int(tracker_id),  # Convert to standard Python int
                    "gender": gender,
                    "age": age,
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence),  # Convert to standard Python float
                    "class_id": int(class_id)  # Convert to standard Python int
                }

            # Track order of area entry
            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    detection_data[tracker_id]["entry_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    detection_data[tracker_id]["exit_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Determine if entering or exiting
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []  # Reset state after count
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []  # Reset state after count

            total_count = entering_count + exiting_count

            # Update bbox history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),  # Convert to standard Python int
                "bbox": [float(x1), float(y1), float(x2), float(y2)],  # Convert to standard Python floats
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

            # Draw bounding box and ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            label = f"ID: {tracker_id} | {gender}, {age}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Draw bottom-right corner marker
            cv2.circle(frame, bottom_right, 5, (0, 0, 255), -1)

        # Draw polygons on the frame
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display counts
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save frame to video
        sink.write_frame(frame)

# Prepare JSON data
json_data = {
    "summary": {
        "total_people": int(total_count),  # Convert to standard Python int
        "total_entering": int(entering_count),  # Convert to standard Python int
        "total_exiting": int(exiting_count),  # Convert to standard Python int
        "total_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "detections": {
        # Convert NumPy int64 keys to standard Python int
        int(tracker_id): data
        for tracker_id, data in detection_data.items()
    }
}

# Save all data to a single JSON file
json_path = "detection_data.json"
with open(json_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Processing complete. Output video saved.")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 138.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 97.0ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 52.0ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 52.0ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 52.0ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 52.0ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 52.0ms
age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:14<00:00, 37.0MB/s]


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:06<00:00, 88.4MB/s]
Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.31it/s]

video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 63.2ms


...


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.47it/s]

video 1/1 (frame 591/592) /content/people-counting.mp4: 384x640 1 potted plant, 63.1ms


video 1/1 (frame 592/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 63.1ms


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.53it/s]

Speed: 4.0ms preprocess, 61.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Processing complete. Output video saved.
Total: 5, Entering: 4, Exiting: 1


#### Carrying Objects

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime
from deepface import DeepFace

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video.mp4"
JSON_OUTPUT_PATH = "detection_data_new.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}
detection_data = {}

# ===== Helper Functions =====
def predict_age_gender(face_img):
    """Predict age and gender using DeepFace"""
    try:
        analysis = DeepFace.analyze(face_img, actions=["age", "gender"], enforce_detection=False)
        return analysis[0]["dominant_gender"], analysis[0]["age"]
    except Exception:
        return "Unknown", "Unknown"

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox

    # Object center should be in lower 40% of person's height
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6

    # Check if object is mostly inside person's bbox
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold

    return overlap and in_carry_position

# ===== Main Processing =====
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections into people and objects
        people = []
        objects = []

        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person
        for person_bbox, confidence, tracker_id in people:
            x1, y1, x2, y2 = person_bbox
            bottom_right = (int(x2), int(y2))

            # Age/gender detection
            face_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = predict_age_gender(face_roi)

            # Check for carried items
            carried_items = {"bag": False, "cat": False, "dog": False}
            for obj_bbox, _, class_id in objects:
                if is_carried(obj_bbox, person_bbox):
                    if class_id in BAG_CLASSES:
                        carried_items["bag"] = True
                    elif class_id == CAT_CLASS:
                        carried_items["cat"] = True
                    elif class_id == DOG_CLASS:
                        carried_items["dog"] = True

            # Initialize or update person data
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": gender,
                    "age": age,
                    "carrying": carried_items,
                    "entry_time": None,
                    "exit_time": None,
                    "bbox_history": [],
                    "confidence": float(confidence)
                }
            else:
                # Update carried items (keep True if previously detected)
                current = detection_data[tracker_id]["carrying"]
                for item in carried_items:
                    if carried_items[item]:
                        current[item] = True
                detection_data[tracker_id]["carrying"] = current

            # Area crossing logic (same as before)
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    detection_data[tracker_id]["entry_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    detection_data[tracker_id]["exit_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Update counts
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            total_count = entering_count + exiting_count

            # Store bounding box history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "carrying": carried_items
            })

            # Prepare annotation text
            carrying_str = []
            if carried_items["bag"]: carrying_str.append("bag")
            if carried_items["cat"]: carrying_str.append("cat")
            if carried_items["dog"]: carrying_str.append("dog")
            carrying_status = " | Carrying: " + ", ".join(carrying_str) if carrying_str else ""

            label = f"ID: {tracker_id} | {gender}, {age}{carrying_status}"

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw counting areas and counters
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)

        cv2.putText(frame, f"Total: {total_count}", (50, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "processing_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 63.2ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 63.1ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 63.2ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 63.2ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 61.8ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 61.8ms


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]

video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 61.8ms



Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.70it/s]

video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 63.3ms


...


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.60it/s]

video 1/1 (frame 591/592) /content/people-counting.mp4: 384x640 1 potted plant, 63.1ms


video 1/1 (frame 592/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 59.0ms


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.63it/s]

Speed: 3.9ms preprocess, 62.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Processing complete. Results saved to output_video.mp4 and detection_data_new.json
Total: 5, Entering: 4, Exiting: 1


#### Get metadata

In [8]:
!pip install ffmpeg-python

##### Get video creation date

In [ ]:
import ffmpeg

def get_video_creation_time(video_path):
    try:
        probe = ffmpeg.probe(video_path)

        # Check both 'format' and 'streams' for creation_time
        creation_time = None

        # 1. Check format tags
        if 'format' in probe and 'tags' in probe['format']:
            creation_time = probe['format']['tags'].get('creation_time')

        # 2. Fallback to stream tags
        if not creation_time:
            for stream in probe.get('streams', []):
                if 'tags' in stream and 'creation_time' in stream['tags']:
                    creation_time = stream['tags']['creation_time']
                    break

        return creation_time

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Usage
creation_time = get_video_creation_time("/content/people-counting.mp4")
if creation_time:
    print(f"Creation time: {creation_time}")
else:
    print("No creation time found in metadata")

Creation time: 2021-01-29T17:53:46.000000Z


##### Get all metadata

In [ ]:
import ffmpeg
from datetime import datetime, timezone
import json

def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Example Usage
metadata = extract_video_metadata("/content/people-counting.mp4")
print(json.dumps(metadata, indent=4))

{
    "filename": "/content/people-counting.mp4",
    "format_name": "mov,mp4,m4a,3gp,3g2,mj2",
    "format_long_name": "QuickTime / MOV",
    "duration_seconds": 19.753333,
    "size_bytes": 42527844,
    "bitrate": 17223561,
    "creation_time": "2021-01-29T17:53:46.000000Z",
    "encoder": null,
    "video_codec": "h264",
    "width": 3840,
    "height": 2160,
    "fps": 29.97002997002997,
    "device_model": null,
    "software": null,
    "audio_codec": "aac",
    "sample_rate": 48000,
    "channels": 2,
    "creation_time_utc": "2021-01-29 17:53:46 UTC",
    "creation_time_local": "2021-01-29 17:53:46 UTC"
}


### Update Video Metadata and Carrying Object Order

In [11]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_updated.mp4"
JSON_OUTPUT_PATH = "detection_data_updated.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}
detection_data = {}

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
                metadata["recording_time"] = metadata["creation_time_local"]  # For backward compatibility
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def predict_age_gender(face_img):
    """Predict age and gender using DeepFace"""
    try:
        analysis = DeepFace.analyze(face_img, actions=["age", "gender"], enforce_detection=False)
        return analysis[0]["dominant_gender"], analysis[0]["age"]
    except Exception:
        return "Unknown", "Unknown"

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person
        for person_bbox, confidence, tracker_id in people:
            x1, y1, x2, y2 = person_bbox
            bottom_right = (int(x2), int(y2))

            # Age/gender detection
            face_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = predict_age_gender(face_roi)

            # Check for carried items
            carried_items = []
            for obj_bbox, _, class_id in objects:
                if is_carried(obj_bbox, person_bbox):
                    if class_id in BAG_CLASSES:
                        carried_items.append("bag")
                    elif class_id == CAT_CLASS:
                        carried_items.append("cat")
                    elif class_id == DOG_CLASS:
                        carried_items.append("dog")

            # Initialize or update person data
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": gender,
                    "age": age,
                    "carrying": carried_items if carried_items else "no objects",
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "bbox_history": [],
                    "confidence": float(confidence)
                }

            # Area crossing logic
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["entry_time"] = entry_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["entry_frame"] = frame_number

            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["exit_time"] = exit_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["exit_frame"] = frame_number

            # Update counts
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            total_count = entering_count + exiting_count

            # Store bounding box history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": (recording_time + timedelta(seconds=frame_number / video_info.fps)).strftime("%Y-%m-%d %H:%M:%S"),
                "carrying": carried_items if carried_items else "no objects"
            })

            # Draw bounding box and label
            carrying_str = ", ".join(carried_items) if carried_items else "no objects"
            label = f"ID: {tracker_id} | {gender}, {age} | Carrying: {carrying_str}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw counting areas and counters
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 66.9ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 67.9ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 63.2ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 72.8ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 84.2ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 68.9ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 65.6ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 74.2ms


...


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.65it/s]

video 1/1 (frame 591/592) /content/people-counting.mp4: 384x640 1 potted plant, 63.1ms


video 1/1 (frame 592/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 63.0ms


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  2.15it/s]

Speed: 4.1ms preprocess, 62.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Processing complete. Results saved to output_video_updated.mp4 and detection_data_updated.json
Total: 5, Entering: 4, Exiting: 1


##### Tracking when only entering and exiting

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_updated.mp4"
JSON_OUTPUT_PATH = "detection_data_updated.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# ===== Initialize =====
model = YOLO("yolov8n.pt")  # Initialize YOLO model
entering_count = 0
exiting_count = 0
tracker_states = {}  # Tracks the state of each person (which areas they've been in)
detection_data = {}  # Stores detailed information about each detection

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract video metadata using ffmpeg"""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {
            "duration": float(probe["format"]["duration"]),
            "width": int(probe["streams"][0]["width"]),
            "height": int(probe["streams"][0]["height"]),
            "fps": eval(probe["streams"][0]["avg_frame_rate"]),
            "creation_time": probe["format"]["tags"].get("creation_time", "unknown")
        }
        return metadata
    except Exception as e:
        print(f"Error extracting metadata: {e}")
        return None

def predict_age_gender(face_img):
    """Predict age and gender using DeepFace"""
    try:
        analysis = DeepFace.analyze(face_img, actions=["age", "gender"], enforce_detection=False)
        return analysis[0]["dominant_gender"], analysis[0]["age"]
    except Exception:
        return "Unknown", "Unknown"

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def convert_to_serializable(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    if isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, np.float32):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime):
        return obj.strftime("%Y-%m-%d %H:%M:%S")
    elif isinstance(obj, dict):
        return {str(k): convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    else:
        return obj

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = int(tracker_ids[i]) if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person
        for person_bbox, confidence, tracker_id in people:
            if tracker_id is None:
                continue

            x1, y1, x2, y2 = person_bbox
            bottom_right = (int(x2), int(y2))

            # Age/gender detection
            face_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = predict_age_gender(face_roi)

            # Check for carried items
            carried_items = []
            for obj_bbox, _, class_id in objects:
                if is_carried(obj_bbox, person_bbox):
                    if class_id in BAG_CLASSES:
                        carried_items.append("bag")
                    elif class_id == CAT_CLASS:
                        carried_items.append("cat")
                    elif class_id == DOG_CLASS:
                        carried_items.append("dog")

            # Initialize or update person data
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                detection_data[tracker_id] = {
                    "tracker_id": tracker_id,
                    "gender": gender,
                    "age": age,
                    "carrying": carried_items if carried_items else "no objects",
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "bbox_history": [],
                    "confidence": float(confidence)
                }

            # Area crossing logic
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["entry_time"] = entry_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["entry_frame"] = frame_number

            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["exit_time"] = exit_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["exit_frame"] = frame_number

            # Update counts
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            # Store bounding box history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": (recording_time + timedelta(seconds=frame_number / video_info.fps)).strftime("%Y-%m-%d %H:%M:%S"),
                "carrying": carried_items if carried_items else "no objects"
            })

            # Draw bounding box and label
            carrying_str = ", ".join(carried_items) if carried_items else "no objects"
            label = f"ID: {tracker_id} | {gender}, {age} | Carrying: {carrying_str}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw counting areas and counters
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_entering": entering_count,
        "total_exiting": exiting_count,
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

# Convert all numpy types to native Python types before JSON serialization
json_output = convert_to_serializable(json_output)

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Entering: {entering_count}, Exiting: {exiting_count}")

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


100%|██████████| 6.25M/6.25M [00:00<00:00, 58.1MB/s]


NameError: name 'SOURCE_VIDEO_PATH' is not defined

### mask and speed

In [9]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


fatal: destination path 'Face-Mask-Detection' already exists and is not an empty directory.


In [10]:
from tensorflow.keras.utils import img_to_array


In [11]:
from tensorflow.keras.models import load_model

model_path = "Face-Mask-Detection/mask_detector.model"
mask_model = load_model(model_path)


In [21]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_mask_speed.mp4"
JSON_OUTPUT_PATH = "detection_data_mask_speed.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}
detection_data = {}
previous_positions = {}  # To track movement between frames

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
                metadata["recording_time"] = metadata["creation_time_local"]  # For backward compatibility
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def predict_age_gender(face_img):
    """Predict age and gender using DeepFace"""
    try:
        analysis = DeepFace.analyze(face_img, actions=["age", "gender"], enforce_detection=False)
        return analysis[0]["dominant_gender"], analysis[0]["age"]
    except Exception:
        return "Unknown", "Unknown"

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def calculate_speed(prev_position, current_position, fps, frame_interval=1):
    """
    Calculate speed in pixels per second between two positions
    frame_interval: number of frames between measurements (for smoothing)
    """
    if prev_position is None or current_position is None:
        return 0

    # Use center points for speed calculation
    prev_center = ((prev_position[0] + prev_position[2]) / 2, (prev_position[1] + prev_position[3]) / 2)
    curr_center = ((current_position[0] + current_position[2]) / 2, (current_position[1] + current_position[3]) / 2)

    # Calculate Euclidean distance
    distance = math.sqrt((curr_center[0] - prev_center[0])**2 + (curr_center[1] - prev_center[1])**2)

    # Convert to pixels per second
    speed = distance * fps / frame_interval
    return speed

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person
        for person_bbox, confidence, tracker_id in people:
            x1, y1, x2, y2 = person_bbox
            bottom_right = (int(x2), int(y2))

            # Calculate speed
            current_position = (x1, y1, x2, y2)
            prev_position = previous_positions.get(tracker_id, None)
            speed = calculate_speed(prev_position, current_position, video_info.fps) if frame_number > 0 else 0
            previous_positions[tracker_id] = current_position

            # Age/gender detection
            face_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = predict_age_gender(face_roi)

            # Mask detection
            mask_status,mask_confidence = detect_mask(face_roi)  # Check if the person is wearing a mask

            # Check for carried items
            carried_items = []
            for obj_bbox, _, class_id in objects:
                if is_carried(obj_bbox, person_bbox):
                    if class_id in BAG_CLASSES:
                        carried_items.append("bag")
                    elif class_id == CAT_CLASS:
                        carried_items.append("cat")
                    elif class_id == DOG_CLASS:
                        carried_items.append("dog")

            # Initialize or update person data
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": gender,
                    "age": age,
                    "carrying": carried_items if carried_items else "no objects",
                    "mask_status":mask_status,
                    "mask_confidence":mask_confidence,
                    "bbox_history": [],
                    "confidence": float(confidence)
                }

            # Area crossing logic
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["entry_time"] = entry_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["entry_frame"] = frame_number
                    # Mask detection
                    mask_status,mask_confidence = detect_mask(face_roi)  # Check if the person is wearing a mask
                    detection_data[tracker_id]["entry_mask_status"] = mask_status
                    detection_data[tracker_id]["entry_mask_confidence"] = mask_confidence


            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["exit_time"] = exit_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["exit_frame"] = frame_number
                    # Mask detection
                    mask_status,mask_confidence = detect_mask(face_roi)  # Check if the person is wearing a mask
                    detection_data[tracker_id]["exit_mask_status"] = mask_status
                    detection_data[tracker_id]["exit_mask_confidence"] = mask_confidence

            # Update counts
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            total_count = entering_count + exiting_count

            # Store bounding box history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": (recording_time + timedelta(seconds=frame_number / video_info.fps)).strftime("%Y-%m-%d %H:%M:%S"),
                "carrying": carried_items if carried_items else "no objects",
                "speed": speed,
                "mask_status":mask_status,
                "mask_confidence":mask_confidence
            })

            # Draw bounding box and label
            carrying_str = ", ".join(carried_items) if carried_items else "no objects"
            label = f"ID: {tracker_id} | {gender}, {age} | Mask: {mask_status} | Carrying: {carrying_str} | Speed: {speed:.2f} px/s"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 3)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw counting areas and counters
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")



video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 4391.6ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 5494.2ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 4120.5ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 bench, 4108.3ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 5401.1ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]

1/1 [==============================] - 0s 59ms/step


video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 2461.3ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.66it/s]

...

1/1 [==============================] - 0s 58ms/step


video 1/1 (frame 591/592) /content/people-counting.mp4: 384x640 1 potted plant, 2508.9ms
video 1/1 (frame 592/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2437.5ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.65it/s]

1/1 [==============================] - 0s 74ms/step


Speed: 5.8ms preprocess, 2729.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processing complete. Results saved to output_video_mask_speed.mp4 and detection_data_mask_speed.json
Total: 5, Entering: 4, Exiting: 1


### restricted

In [32]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_restrcted.mp4"
JSON_OUTPUT_PATH = "detection_data_restrcted.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Add a restricted area (randomly chosen coordinates - adjust as needed)
restricted_area = np.array([(500, 500), (800, 500), (800, 800), (500, 800)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
restricted_area_count = 0  # New counter for restricted area
tracker_states = {}
restricted_area_states = {}  # To track who entered restricted area
detection_data = {}
previous_positions = {}  # To track movement between frames

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
                metadata["recording_time"] = metadata["creation_time_local"]  # For backward compatibility
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def predict_age_gender(face_img):
    """Predict age and gender using DeepFace"""
    try:
        analysis = DeepFace.analyze(face_img, actions=["age", "gender"], enforce_detection=False)
        return analysis[0]["dominant_gender"], analysis[0]["age"]
    except Exception:
        return "Unknown", "Unknown"

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def calculate_speed(prev_position, current_position, fps, frame_interval=1):
    """
    Calculate speed in pixels per second between two positions
    frame_interval: number of frames between measurements (for smoothing)
    """
    if prev_position is None or current_position is None:
        return 0

    # Use center points for speed calculation
    prev_center = ((prev_position[0] + prev_position[2]) / 2, (prev_position[1] + prev_position[3]) / 2)
    curr_center = ((current_position[0] + current_position[2]) / 2, (current_position[1] + current_position[3]) / 2)

    # Calculate Euclidean distance
    distance = math.sqrt((curr_center[0] - prev_center[0])**2 + (curr_center[1] - prev_center[1])**2)

    # Convert to pixels per second
    speed = distance * fps / frame_interval
    return speed

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person
        for person_bbox, confidence, tracker_id in people:
            x1, y1, x2, y2 = person_bbox
            bottom_right = (int(x2), int(y2))
            center = (int((x1 + x2) / 2), int((y1 + y2) / 2))

            # Calculate speed
            current_position = (x1, y1, x2, y2)
            prev_position = previous_positions.get(tracker_id, None)
            speed = calculate_speed(prev_position, current_position, video_info.fps) if frame_number > 0 else 0
            previous_positions[tracker_id] = current_position

            # Age/gender detection
            face_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            gender, age = predict_age_gender(face_roi)

            # Mask detection
            mask_status, mask_confidence = detect_mask(face_roi)

            # Check for carried items
            carried_items = []
            for obj_bbox, _, class_id in objects:
                if is_carried(obj_bbox, person_bbox):
                    if class_id in BAG_CLASSES:
                        carried_items.append("bag")
                    elif class_id == CAT_CLASS:
                        carried_items.append("cat")
                    elif class_id == DOG_CLASS:
                        carried_items.append("dog")

            # Initialize or update person data
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = []
                restricted_area_states[tracker_id] = False
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": gender,
                    "age": age,
                    "carrying": carried_items if carried_items else "no objects",
                    "mask_status": mask_status,
                    "mask_confidence": mask_confidence,
                    "bbox_history": [],
                    "confidence": float(confidence),
                    "entered_restricted_area": False,
                    "restricted_area_entry_time": None,
                    "restricted_area_entry_frame": None
                }

            # Check restricted area entry
            in_restricted_area = cv2.pointPolygonTest(restricted_area, center, False) >= 0
            if in_restricted_area and not restricted_area_states[tracker_id]:
                restricted_area_states[tracker_id] = True
                detection_data[tracker_id]["entered_restricted_area"] = True
                detection_data[tracker_id]["restricted_area_entry_time"] = (
                    recording_time + timedelta(seconds=frame_number / video_info.fps)).strftime("%Y-%m-%d %H:%M:%S")
                detection_data[tracker_id]["restricted_area_entry_frame"] = frame_number
                restricted_area_count += 1
            elif not in_restricted_area and restricted_area_states[tracker_id]:
                restricted_area_states[tracker_id] = False

            # Area crossing logic
            in_area1 = cv2.pointPolygonTest(area1, bottom_right, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_right, False) >= 0

            if in_area2 and "area2" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area2")
                if tracker_states[tracker_id] == ["area2"]:
                    entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["entry_time"] = entry_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["entry_frame"] = frame_number
                    detection_data[tracker_id]["entry_mask_status"] = mask_status
                    detection_data[tracker_id]["entry_mask_confidence"] = mask_confidence

            if in_area1 and "area1" not in tracker_states[tracker_id]:
                tracker_states[tracker_id].append("area1")
                if tracker_states[tracker_id] == ["area1"]:
                    exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                    detection_data[tracker_id]["exit_time"] = exit_time.strftime("%Y-%m-%d %H:%M:%S")
                    detection_data[tracker_id]["exit_frame"] = frame_number
                    detection_data[tracker_id]["exit_mask_status"] = mask_status
                    detection_data[tracker_id]["exit_mask_confidence"] = mask_confidence

            # Update counts
            if tracker_states[tracker_id] == ["area2", "area1"]:
                entering_count += 1
                tracker_states[tracker_id] = []
            elif tracker_states[tracker_id] == ["area1", "area2"]:
                exiting_count += 1
                tracker_states[tracker_id] = []

            total_count = entering_count + exiting_count

            # Store bounding box history
            detection_data[tracker_id]["bbox_history"].append({
                "frame_number": int(frame_number),
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "timestamp": (recording_time + timedelta(seconds=frame_number / video_info.fps)).strftime("%Y-%m-%d %H:%M:%S"),
                "carrying": carried_items if carried_items else "no objects",
                "speed": speed,
                "mask_status": mask_status,
                "mask_confidence": mask_confidence,
                "in_restricted_area": in_restricted_area
            })

            # Draw bounding box and label
            carrying_str = ", ".join(carried_items) if carried_items else "no objects"
            label = f"ID: {tracker_id} | {gender}, {age} | Mask: {mask_status} | Carrying: {carrying_str} | Speed: {speed:.2f} px/s"
            if in_restricted_area:
                label += " | IN RESTRICTED AREA"
                box_color = (0, 0, 255)  # Red for restricted area
            else:
                box_color = (255, 0, 0)  # Blue for normal

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 3)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw counting areas and counters
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.polylines(frame, [restricted_area], isClosed=True, color=(0, 0, 255), thickness=2)
        cv2.putText(frame, "Restricted Area", (restricted_area[0][0], restricted_area[0][1] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Display counters
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Restricted Area Entries: {restricted_area_count}", (50, 200),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 2)

        # Write frame to output video
        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "restricted_area_entries": int(restricted_area_count),  # New field
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")
print(f"Restricted Area Entries: {restricted_area_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 4880.6ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4461.6ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4065.5ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 5561.8ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4042.0ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 4021.8ms


Action: gender: 100%|██████████| 2/2 [00:02<00:00,  1.17s/it]


1/1 [==============================] - 0s 173ms/step
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3156.5ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.66it/s]

1/1 [==============================] - 0s 58ms/step


...

1/1 [==============================] - 0s 56ms/step


video 1/1 (frame 591/592) /content/people-counting.mp4: 384x640 1 potted plant, 2444.4ms
video 1/1 (frame 592/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2436.1ms


Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.67it/s]

1/1 [==============================] - 0s 55ms/step
Speed: 5.9ms preprocess, 2696.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing complete. Results saved to output_video_restrcted.mp4 and detection_data_mask_restrcted.json
Total: 5, Entering: 4, Exiting: 1
Restricted Area Entries: 0


### tracking when entering and exiting

In [33]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_tracking.mp4"
JSON_OUTPUT_PATH = "detection_data_tracking.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Tracks area crossings
detection_data = {}  # Stores all person data
mask_model = load_model("Face-Mask-Detection/mask_detector.model")

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract video metadata using ffmpeg-python"""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
            })

            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                })

        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),
            })

        return metadata
    except Exception as e:
        print(f"Metadata error: {str(e)}")
        return None

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def analyze_person(frame, bbox, objects):
    """Analyze a person's attributes at entry/exit points"""
    x1, y1, x2, y2 = map(int, bbox)
    face_roi = frame[y1:y2, x1:x2]

    # Age/gender detection
    try:
        analysis = DeepFace.analyze(face_roi, actions=['age', 'gender'], enforce_detection=False)
        gender = analysis[0]['dominant_gender']
        age = analysis[0]['age']
    except:
        gender, age = "Unknown", "Unknown"

    # Mask detection
    mask_status, mask_conf = detect_mask(face_roi)

    # Check for carried items
    carried_items = []
    for obj_bbox, _, class_id in objects:
        if is_carried(obj_bbox, bbox):
            if class_id in BAG_CLASSES:
                carried_items.append("bag")
            elif class_id == CAT_CLASS:
                carried_items.append("cat")
            elif class_id == DOG_CLASS:
                carried_items.append("dog")

    return gender, age, mask_status, mask_conf, carried_items if carried_items else "no objects"

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person's area crossings
        for bbox, conf, tracker_id in people:
            x1, y1, x2, y2 = bbox
            bottom_center = (int((x1+x2)/2), int(y2))

            # Initialize tracker state if new person
            if tracker_id not in tracker_states:
                tracker_states[tracker_id] = {
                    'current_area': None,
                    'last_area': None,
                    'entry_time': None
                }
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": "Unknown",
                    "age": "Unknown",
                    "carrying": "none",
                    "mask_status": "unknown",
                    "mask_confidence": 0.0,
                    "bbox_history": [],
                    "confidence": float(conf),
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "entry_mask_status": None,
                    "exit_mask_status": None
                }

            # Check area crossings
            in_area1 = cv2.pointPolygonTest(area1, bottom_center, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_center, False) >= 0

            current_area = None
            if in_area1: current_area = "area1"
            elif in_area2: current_area = "area2"

            # Handle area transitions
            if current_area != tracker_states[tracker_id]['current_area']:
                # Entry event (area2)
                if current_area == "area2" and tracker_states[tracker_id]['last_area'] != "area2":
                    # Analyze person attributes
                    gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                    # Update detection data
                    detection_data[tracker_id].update({
                        "gender": gender,
                        "age": age,
                        "carrying": carrying,
                        "mask_status": mask_status,
                        "mask_confidence": mask_conf,
                        "entry_time": (datetime.now(timezone.utc)).strftime("%Y-%m-%d %H:%M:%S"),
                        "entry_frame": frame_number,
                        "entry_mask_status": mask_status,
                        "entry_mask_confidence": mask_conf
                    })
                    entering_count += 1
                    total_count += 1

                # Exit event (area1 after area2)
                elif current_area == "area1" and tracker_states[tracker_id]['last_area'] == "area2":
                    # Analyze person attributes again at exit
                    _, _, mask_status, mask_conf, _ = analyze_person(frame, bbox, objects)

                    # Update detection data
                    detection_data[tracker_id].update({
                        "exit_time": (datetime.now(timezone.utc)).strftime("%Y-%m-%d %H:%M:%S"),
                        "exit_frame": frame_number,
                        "exit_mask_status": mask_status,
                        "exit_mask_confidence": mask_conf
                    })
                    exiting_count += 1
                    total_count += 1

                # Update tracker state
                tracker_states[tracker_id]['last_area'] = tracker_states[tracker_id]['current_area']
                tracker_states[tracker_id]['current_area'] = current_area

            # Draw bounding box if person is being tracked
            if detection_data[tracker_id]['entry_time']:
                person = detection_data[tracker_id]
                label = f"ID: {tracker_id} | {person['gender']}, {person['age']} | Mask: {person['mask_status']}"
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # Draw counters and areas
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
        if data['entry_time'] is not None  # Only include people who actually entered
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 5433.6ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4189.8ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4801.5ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4620.7ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 5071.0ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3176.9ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2974.6ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2473.6ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2488.1ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 1 potted plant, 2465.7ms
video 

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


1/1 [==============================] - 1s 1s/step
video 1/1 (frame 56/592) /content/people-counting.mp4: 384x640 4 persons, 1 bench, 1 potted plant, 2529.3ms
video 1/1 (frame 57/592) /content/people-counting.mp4: 384x640 4 persons, 1 bench, 1 potted plant, 2443.5ms
video 1/1 (frame 58/592) /content/people-counting.mp4: 384x640 4 persons, 1 car, 1 bench, 1 potted plant, 2469.6ms
video 1/1 (frame 59/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2820.3ms
video 1/1 (frame 60/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 3091.6ms
video 1/1 (frame 61/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2486.5ms
video 1/1 (frame 62/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2472.0ms
video 1/1 (frame 63/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2471.1ms
video 1/1 (frame 64/592) /content/people-counting.mp4: 384x640 3 persons, 1 car

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.67it/s]

1/1 [==============================] - 0s 57ms/step


...

1/1 [==============================] - 0s 93ms/step


video 1/1 (frame 424/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2893.3ms
video 1/1 (frame 425/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2471.4ms
video 1/1 (frame 426/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2513.4ms
video 1/1 (frame 427/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2449.2ms
video 1/1 (frame 428/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3465.3ms
video 1/1 (frame 429/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2457.4ms
video 1/1 (frame 430/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2468.8ms
video 1/1 (frame 431/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2468.2ms
video 1/1 (frame 432/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2790.0ms
video 1/1 (frame 433/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3014.8ms
video 1/

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.60it/s]

1/1 [==============================] - 0s 58ms/step


video 1/1 (frame 539/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2457.7ms
video 1/1 (frame 540/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2407.2ms
video 1/1 (frame 541/592) /content/people-counting.mp4: 384x640 1 person, 3040.2ms
video 1/1 (frame 542/592) /content/people-counting.mp4: 384x640 1 person, 2830.8ms
video 1/1 (frame 543/592) /content/people-counting.mp4: 384x640 1 person, 2426.4ms
video 1/1 (frame 544/592) /content/people-counting.mp4: 384x640 1 person, 2462.3ms
video 1/1 (frame 545/592) /content/people-counting.mp4: 384x640 1 person, 2472.4ms
video 1/1 (frame 546/592) /content/people-counting.mp4: 384x640 1 person, 3414.2ms
video 1/1 (frame 547/592) /content/people-counting.mp4: 384x640 1 person, 2488.9ms
video 1/1 (frame 548/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2468.2ms
video 1/1 (frame 549/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2423.4ms
video 1/1 (frame 550/59

#### next

In [35]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_final_tracking3.mp4"
JSON_OUTPUT_PATH = "detection_data_final_tracking3.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
tracker_states = {}  # Tracks area crossings
detection_data = {}  # Stores entry/exit information

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract video metadata using ffmpeg-python"""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
            })

            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                })

        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),
            })

        return metadata
    except Exception as e:
        print(f"Metadata error: {str(e)}")
        return None

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def analyze_person(frame, bbox, objects):
    """Analyze a person's attributes at entry/exit points"""
    x1, y1, x2, y2 = map(int, bbox)
    face_roi = frame[y1:y2, x1:x2]

    # Age/gender detection
    try:
        analysis = DeepFace.analyze(face_roi, actions=['age', 'gender'], enforce_detection=False)
        gender = analysis[0]['dominant_gender']
        age = analysis[0]['age']
    except:
        gender, age = "Unknown", "Unknown"

    # Mask detection
    mask_status, mask_conf = detect_mask(face_roi)

    # Check for carried items
    carried_items = []
    for obj_bbox, _, class_id in objects:
        if is_carried(obj_bbox, bbox):
            if class_id in BAG_CLASSES:
                carried_items.append("bag")
            elif class_id == CAT_CLASS:
                carried_items.append("cat")
            elif class_id == DOG_CLASS:
                carried_items.append("dog")

    return gender, age, mask_status, mask_conf, carried_items if carried_items else "no objects"

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person's area crossings
        for bbox, conf, tracker_id in people:
            x1, y1, x2, y2 = bbox
            bottom_center = (int((x1+x2)/2), int(y2))

            # Check area crossings
            in_area1 = cv2.pointPolygonTest(area1, bottom_center, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_center, False) >= 0

            current_area = None
            if in_area1: current_area = "area1"
            elif in_area2: current_area = "area2"

            # Initialize tracker only when entering monitored areas
            if tracker_id not in tracker_states and current_area:
                tracker_states[tracker_id] = {
                    'current_area': current_area,
                    'last_area': None,
                    'entry_time': None
                }
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": "Unknown",
                    "age": "Unknown",
                    "carrying": "none",
                    "mask_status": "unknown",
                    "mask_confidence": 0.0,
                    "confidence": float(conf),
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "entry_mask_status": None,
                    "exit_mask_status": None
                }

            # Only process if person is in our tracking system (entered monitored area)
            if tracker_id in tracker_states:
                # Handle area transitions
                if current_area != tracker_states[tracker_id]['current_area']:
                    # Entry event (area2)
                    if current_area == "area1" and tracker_states[tracker_id]['last_area'] == "area2":
                        # Analyze person attributes
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "entry_time": entry_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "entry_frame": frame_number,
                            "entry_mask_status": mask_status,
                            "entry_mask_confidence": mask_conf
                        })
                        entering_count+= 1
                        total_count += 1

                    # Exit event (area1 after area2)
                    elif current_area == "area2" and tracker_states[tracker_id]['last_area'] == "area1":
                        # Analyze person attributes again at exit
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "exit_time": exit_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "exit_frame": frame_number,
                            "exit_mask_status": mask_status,
                            "exit_mask_confidence": mask_conf
                        })
                        exiting_count += 1
                        total_count += 1

                    # Update tracker state
                    tracker_states[tracker_id]['last_area'] = tracker_states[tracker_id]['current_area']
                    tracker_states[tracker_id]['current_area'] = current_area

                # Draw bounding box if person is being tracked
                if detection_data[tracker_id]['entry_time']:
                    person = detection_data[tracker_id]
                    label = f"ID: {tracker_id} | {person['gender']}, {person['age']} | Mask: {person['mask_status']}"
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # Draw counters and areas
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 4739.4ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4735.1ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4116.2ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 5631.0ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4501.4ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3902.3ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3165.3ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2769.0ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 2446.0ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 1 potted plant, 2462.3ms
video 

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.57it/s]

1/1 [==============================] - 0s 59ms/step


video 1/1 (frame 65/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3486.0ms
video 1/1 (frame 66/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2439.6ms
video 1/1 (frame 67/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2414.6ms
video 1/1 (frame 68/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2442.1ms
video 1/1 (frame 69/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2656.0ms
video 1/1 (frame 70/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3175.1ms
video 1/1 (frame 71/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2444.3ms
video 1/1 (frame 72/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2438.9ms
video 1/1 (frame 73/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 po

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


...

1/1 [==============================] - 0s 63ms/step


video 1/1 (frame 424/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2420.7ms
video 1/1 (frame 425/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3410.4ms
video 1/1 (frame 426/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2458.1ms
video 1/1 (frame 427/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2431.0ms
video 1/1 (frame 428/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2400.7ms
video 1/1 (frame 429/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2936.4ms
video 1/1 (frame 430/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2929.2ms
video 1/1 (frame 431/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2425.6ms
video 1/1 (frame 432/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2419.3ms
video 1/1 (frame 433/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2452.3ms
video 1/

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.60it/s]

1/1 [==============================] - 0s 60ms/step


video 1/1 (frame 539/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3498.6ms
video 1/1 (frame 540/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2425.0ms
video 1/1 (frame 541/592) /content/people-counting.mp4: 384x640 1 person, 2443.9ms
video 1/1 (frame 542/592) /content/people-counting.mp4: 384x640 1 person, 2448.3ms
video 1/1 (frame 543/592) /content/people-counting.mp4: 384x640 1 person, 2530.6ms
video 1/1 (frame 544/592) /content/people-counting.mp4: 384x640 1 person, 3321.1ms
video 1/1 (frame 545/592) /content/people-counting.mp4: 384x640 1 person, 2483.6ms
video 1/1 (frame 546/592) /content/people-counting.mp4: 384x640 1 person, 2435.8ms
video 1/1 (frame 547/592) /content/people-counting.mp4: 384x640 1 person, 2475.1ms
video 1/1 (frame 548/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3169.5ms
video 1/1 (frame 549/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2653.1ms
video 1/1 (frame 550/59

In [13]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_final_trackingID_based.mp4"
JSON_OUTPUT_PATH = "detection_data_final_trackingID_based.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Define restricted area (example coordinates - adjust as needed)
restricted_area = np.array([(500, 500), (800, 500), (800, 800), (500, 800)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
restricted_area_count = 0  # New counter for restricted area
tracker_states = {}  # Tracks area crossings
detection_data = {}  # Stores entry/exit information
restricted_people = set()  # Track people who entered restricted area

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
                metadata["recording_time"] = metadata["creation_time_local"]  # For backward compatibility
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def analyze_person(frame, bbox, objects):
    """Analyze a person's attributes at entry/exit points"""
    x1, y1, x2, y2 = map(int, bbox)
    face_roi = frame[y1:y2, x1:x2]

    # Age/gender detection
    try:
        analysis = DeepFace.analyze(face_roi, actions=['age', 'gender'], enforce_detection=False)
        gender = analysis[0]['dominant_gender']
        age = analysis[0]['age']
    except:
        gender, age = "Unknown", "Unknown"

    # Mask detection
    mask_status, mask_conf = detect_mask(face_roi)

    # Check for carried items
    carried_items = []
    for obj_bbox, _, class_id in objects:
        if is_carried(obj_bbox, bbox):
            if class_id in BAG_CLASSES:
                carried_items.append("bag")
            elif class_id == CAT_CLASS:
                carried_items.append("cat")
            elif class_id == DOG_CLASS:
                carried_items.append("dog")

    return gender, age, mask_status, mask_conf, carried_items if carried_items else "no objects"

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person's area crossings
        for bbox, conf, tracker_id in people:
            x1, y1, x2, y2 = bbox
            bottom_center = (int((x1+x2)/2), int(y2))

            # Check area crossings
            in_area1 = cv2.pointPolygonTest(area1, bottom_center, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_center, False) >= 0
            in_restricted = cv2.pointPolygonTest(restricted_area, bottom_center, False) >= 0

            current_area = None
            if in_area1: current_area = "area1"
            elif in_area2: current_area = "area2"

            # Initialize tracker only when entering monitored areas
            if tracker_id not in tracker_states and current_area:
                tracker_states[tracker_id] = {
                    'current_area': current_area,
                    'last_area': None,
                    'entry_time': None,
                    'entered_restricted': False
                }
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": "Unknown",
                    "age": "Unknown",
                    "carrying": "none",
                    "mask_status": "unknown",
                    "mask_confidence": 0.0,
                    "confidence": float(conf),
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "entry_mask_status": None,
                    "exit_mask_status": None,
                    "entered_restricted": False,
                    "restricted_entry_time": None,
                    "restricted_exit_time": None
                }

            # Only process if person is in our tracking system (entered monitored area)
            if tracker_id in tracker_states:
                # Check restricted area entry
                if in_restricted and not tracker_states[tracker_id]['entered_restricted']:
                    tracker_states[tracker_id]['entered_restricted'] = True
                    detection_data[tracker_id]['entered_restricted'] = True
                    detection_data[tracker_id]['restricted_entry_time'] = (
                        recording_time + timedelta(seconds=frame_number / video_info.fps)
                    ).strftime("%Y-%m-%d %H:%M:%S")
                    restricted_area_count += 1
                    restricted_people.add(tracker_id)

                    # Draw alert for restricted area entry
                    cv2.putText(frame, "RESTRICTED AREA ENTRY!", (int(x1), int(y1)-30),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                # Check restricted area exit
                elif not in_restricted and tracker_states[tracker_id]['entered_restricted']:
                    detection_data[tracker_id]['restricted_exit_time'] = (
                        recording_time + timedelta(seconds=frame_number / video_info.fps)
                    ).strftime("%Y-%m-%d %H:%M:%S")

                # Handle area transitions
                if current_area != tracker_states[tracker_id]['current_area']:
                    # Entry event (area2)
                    if current_area == "area1" and tracker_states[tracker_id]['last_area'] == "area2":
                        # Analyze person attributes
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "entry_time": entry_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "entry_frame": frame_number,
                            "entry_mask_status": mask_status,
                            "entry_mask_confidence": mask_conf
                        })
                        entering_count += 1
                        total_count += 1

                    # Exit event (area1 after area2)
                    elif current_area == "area2" and tracker_states[tracker_id]['last_area'] == "area1":
                        # Analyze person attributes again at exit
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "exit_time": exit_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "exit_frame": frame_number,
                            "exit_mask_status": mask_status,
                            "exit_mask_confidence": mask_conf
                        })
                        exiting_count += 1
                        total_count += 1

                    # Update tracker state
                    tracker_states[tracker_id]['last_area'] = tracker_states[tracker_id]['current_area']
                    tracker_states[tracker_id]['current_area'] = current_area

                # Draw bounding box if person is being tracked
                if detection_data[tracker_id]['entry_time']:
                    person = detection_data[tracker_id]
                    label = f"ID: {tracker_id} | {person['gender']}, {person['age']} | Mask: {person['mask_status']}"
                    if person['entered_restricted']:
                        # Highlight people who entered restricted area
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 3)
                        label += " | RESTRICTED"
                    else:
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # Draw counters and areas
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.polylines(frame, [restricted_area], isClosed=True, color=(0, 0, 255), thickness=2)
        cv2.putText(frame, "RESTRICTED AREA", (restricted_area[0][0], restricted_area[0][1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Restricted Area: {restricted_area_count}", (50, 200),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "restricted_area_entries": int(restricted_area_count),
        "restricted_people_ids": [int(id) for id in restricted_people],
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")
print(f"Restricted area entries: {restricted_area_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 5198.6ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4178.9ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4028.9ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 5365.4ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 3970.3ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 4011.5ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 5307.7ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 4056.9ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3513.2ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 1 potted plant, 2932.0ms
video 

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.62it/s]

1/1 [==============================] - 0s 60ms/step


video 1/1 (frame 65/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2331.2ms
video 1/1 (frame 66/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3317.0ms
video 1/1 (frame 67/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2392.3ms
video 1/1 (frame 68/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2345.5ms
video 1/1 (frame 69/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2363.6ms
video 1/1 (frame 70/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2554.4ms
video 1/1 (frame 71/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3046.7ms
video 1/1 (frame 72/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2373.5ms
video 1/1 (frame 73/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 po

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.68it/s]

...

1/1 [==============================] - 0s 52ms/step


video 1/1 (frame 424/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2349.8ms
video 1/1 (frame 425/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2376.7ms
video 1/1 (frame 426/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2368.0ms
video 1/1 (frame 427/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2513.9ms
video 1/1 (frame 428/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3174.9ms
video 1/1 (frame 429/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2365.0ms
video 1/1 (frame 430/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2398.1ms
video 1/1 (frame 431/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2341.7ms
video 1/1 (frame 432/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2860.4ms
video 1/1 (frame 433/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2759.0ms
video 1/

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.69it/s]

1/1 [==============================] - 0s 53ms/step


video 1/1 (frame 539/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2384.4ms
video 1/1 (frame 540/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2533.2ms
video 1/1 (frame 541/592) /content/people-counting.mp4: 384x640 1 person, 3164.9ms
video 1/1 (frame 542/592) /content/people-counting.mp4: 384x640 1 person, 2371.7ms
video 1/1 (frame 543/592) /content/people-counting.mp4: 384x640 1 person, 2341.0ms
video 1/1 (frame 544/592) /content/people-counting.mp4: 384x640 1 person, 2367.9ms
video 1/1 (frame 545/592) /content/people-counting.mp4: 384x640 1 person, 2898.6ms
video 1/1 (frame 546/592) /content/people-counting.mp4: 384x640 1 person, 2727.6ms
video 1/1 (frame 547/592) /content/people-counting.mp4: 384x640 1 person, 2399.7ms
video 1/1 (frame 548/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2370.6ms
video 1/1 (frame 549/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2366.9ms
video 1/1 (frame 550/59